## Equal Weight S&P 500 IndexFund

### create a Python script that will accept the value of  portfolio and tell how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.


In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

### Acquiring an API Token/ using IEX Cloud

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

### Making the first API call (IEX Cloud)
##### Using sandbox to get randomnized data

In [15]:
symbol='MSFT'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

print(data)

{'symbol': 'MSFT', 'companyName': 'Microsoft Corporation', 'primaryExchange': 'SAAQDN', 'calculationPrice': 'close', 'open': 289.89, 'openTime': 1663118089977, 'openSource': 'foalific', 'close': 299.46, 'closeTime': 1709754159525, 'closeSource': 'iaifoclf', 'high': 304.5, 'highTime': 1700005643359, 'highSource': 'ycr lu peiiae5em 1ddtne', 'low': 289.89, 'lowTime': 1690118455338, 'lowSource': 'tcdimeudi1ene  ap eyrl5', 'latestPrice': 291.33, 'latestSource': 'Close', 'latestTime': 'August 13, 2021', 'latestUpdate': 1660901415786, 'latestVolume': 18506234, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 294.91, 'delayedPriceTime': 1652227729656, 'oddLotDelayedPrice': 298.43, 'oddLotDelayedPriceTime': 1651524797616, 'extendedPrice': 304.8, 'extendedChange': 3.31, 'extendedChangePercent': 0.01189, 'extendedPriceTime': 1631877661909, 'previousClose': 292.32, 'previousVolume': 13971324, 'change': 2.94, 'changePercent': 0.01043, 'volume': 18815641, 'i

### parsing API Call

In [17]:
data['latestPrice']
data['marketCap']

2252031954562

#### making into a dataframe

In [18]:
column = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = column)
final_dataframe

Empty DataFrame
Columns: [Ticker, Price, Market Capitalization, Number Of Shares to Buy]
Index: []

In [20]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = column), 
                                        ignore_index = True)
final_dataframe

Ticker   Price Market Capitalization Number Of Shares to Buy
0   AAPL  291.33         2252031954562                     N/A

In [25]:
dataframe = pd.DataFrame(columns=column)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    dataframe = dataframe.append(
        pd.Series([stock,data['latestPrice'],data['marketCap'],'N/A'],index=column), ignore_index = True
    )
    

In [26]:
dataframe

Ticker   Price Market Capitalization Number Of Shares to Buy
0        A  164.24           48820204829                     N/A
1      AAL   20.75           13325437808                     N/A
2      AAP  209.80           13737062021                     N/A
3     AAPL  153.13         2550140414483                     N/A
4     ABBV  117.52          208175431280                     N/A
..     ...     ...                   ...                     ...
500    YUM  138.62           40861074881                     N/A
501    ZBH  151.64           31081669779                     N/A
502   ZBRA  588.70           31327663412                     N/A
503   ZION   56.99            9127799525                     N/A
504    ZTS  208.19           98908873468                     N/A

[505 rows x 4 columns]

### Calculating the Number of Shares to Buy
As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.



In [27]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:150000000


In [30]:
position_size = float(portfolio_size) / len(dataframe.index)
for i in range(0, len(dataframe['Ticker'])-1):
    dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / dataframe['Price'][i])
dataframe

Ticker   Price Market Capitalization Number Of Shares to Buy
0        A  164.24           48820204829                    1808
1      AAL   20.75           13325437808                   14314
2      AAP  209.80           13737062021                    1415
3     AAPL  153.13         2550140414483                    1939
4     ABBV  117.52          208175431280                    2527
..     ...     ...                   ...                     ...
500    YUM  138.62           40861074881                    2142
501    ZBH  151.64           31081669779                    1958
502   ZBRA  588.70           31327663412                     504
503   ZION   56.99            9127799525                    5211
504    ZTS  208.19           98908873468                     N/A

[505 rows x 4 columns]

### bringing it to excel using xlswriter


In [38]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [39]:

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [40]:

writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)
writer.save()